<a href="https://colab.research.google.com/github/LinusBach/SentimentAnalysis/blob/main/sentiAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple sentiment analysis

Sentiment analysis, using iMDB database

First, implement and train a feedforward NN model with TF-IDF. And then train your
model using word2vec embedding. Report both training and development accuracy on
the dataset. Try to use stochastic gradient descent or (mini-batch) stochastic gradient
descent!

In [11]:
!pip install tensorflow
!pip install nltk
!pip install gensim

[nltk_data] Downloading package stopwords to /Users/uni/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### imports


In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow import keras
import nltk
nltk.download('stopwords')
import csv
import numpy as np
# from sklearn.feature_extraction.text import TfidfVectorizer
import gensim

[nltk_data] Downloading package stopwords to /Users/uni/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### load dataset into memory
return a list of docs and a list of respective labels

In [2]:
def load_data (filename):
  content = list()
  labels = list()

  has_header = True
  # detect if file has a header
  # with open(filename, 'r') as file:
  #   sample = file.read(64)
  #   has_header = csv.Sniffer().has_header(sample)

  with open(filename, 'r') as file:
    reader = csv.reader(file)
    # skip first line if file has a header 
    if has_header:
      next(reader)
    for c, l in reader:
      content.append(c)
      labels.append(l)
  return content, labels

### turn a dataset into clean tokens

In [3]:
def clean_data(data):
  corpus = list()
  corp_voc = dict()
  stop = nltk.corpus.stopwords.words("english")
  # regex tokenizer, find words, numbers and words containing '
  tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+(?:'\w)?")
  for doc in data:
    doc = tokenizer.tokenize(doc)
    doc_cleaned = dict()
    for tok in doc:
      # make all words lower case
      tok = tok.lower()
      # filter out numbers 
      if not tok.isdigit() and tok not in stop:
        # add clean token to document
        if tok in doc_cleaned:
          doc_cleaned[tok] += 1
        else:
          doc_cleaned[tok] = 1
    for tok in doc_cleaned.keys():
    # increase corpus vocabulary
        if tok in corp_voc:
          corp_voc[tok] += 1
        else:
          corp_voc[tok] = 1
    corpus.append(doc_cleaned)
  return corpus, corp_voc

# filter all words out of a corpus that are not in a vocabulary
def get_filtered_corpus(corpus, vocab):
  clean_corpus = list()
  for doc in corpus:
    clean_doc = dict()
    for tok in doc:
      if tok in vocab:
        clean_doc[tok] = doc[tok]
    clean_corpus.append(clean_doc)
  return clean_corpus

### preprocess the dataset

tf-idf vectorizer

In [4]:
# corpus must be a list of dicts of form (token: occurences)
# vocab must be a dict of form (token: documents in corpus containing token)
def preprocess_tf_idf(corpus, vocab):
  processed = np.zeros((len(corpus), len(vocab)))
  idf = get_idf(vocab)
  token_order = {tok: i for i, tok, in enumerate(sorted(vocab.keys()))}
  for n_doc, doc in enumerate(corpus):
    tf = get_tf(doc)
    for tok in set(doc):
      tok_pos = token_order[tok]
      processed[n_doc][tok_pos] = tf[tok]*idf[tok]
  return processed

def get_tf(doc):
  tf = dict()
  for tok, occ in doc.items():
    tf[tok] = occ / len(doc)
  return tf

def get_idf(corp_voc):
  idf = dict()
  for tok, docs_containing in corp_voc.items():
    idf[tok] = np.log10(len(corp_voc) / docs_containing)
  return idf


semantic embeddings
pooling word2vec embeddings

In [5]:
def preprocess_sem_vec(corpus):
  processed = list()
  model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
  total_used = 0
  total_available = 0
  for n_doc, doc in enumerate(corpus):
    vecs = [occ*model[tok] for tok, occ in doc.items() if tok in model.key_to_index]
    total_used += len(vecs)
    total_available += len(doc.items())
    processed.append(np.mean(vecs, axis=0))

  return np.array(processed)

load training data

In [21]:
# read data from file
raw_data, labels = load_data("Train.csv")
full_corpus, full_vocab = clean_data(raw_data)

# select part of vocabulary
frequencies = sorted(full_vocab.items(), key=lambda x : x[1], reverse=True)
# number of most common non-stop words that we consider
VOCAB_SIZE = 20000
# number of most frequent words to be disregarded, but this is now irrelevant due to the removal of stop words
HIGHER_CUTOFF = 0
vocab = {x[0] : x[1] for x in frequencies[HIGHER_CUTOFF:HIGHER_CUTOFF+VOCAB_SIZE]}

corpus = get_filtered_corpus(full_corpus, vocab.keys())
# process labels
taining_labels = np.array([0 if int(label) == 0 else 1 for label in labels]) # keras.utils.to_categorical(labels, num_classes=2)

load validation data

In [7]:
v_raw, v_labels = load_data("Valid.csv")
full_valid_corpus, _ = clean_data(v_raw)
valid_corpus = get_filtered_corpus(full_valid_corpus, vocab)
valid_labels = np.array([0 if int(label) == 0 else 1 for label in v_labels]) # keras.utils.to_categorical(v_labels, num_classes=2)

load testing data

In [8]:
t_raw, t_labels = load_data("Test.csv")
full_test_corpus, _ = clean_data(t_raw)
test_corpus = get_filtered_corpus(full_test_corpus, vocab)
test_labels = np.array([0 if int(label) == 0 else 1 for label in t_labels]) # keras.utils.to_categorical(t_labels, num_classes=2)

define and preprocess training, testing and validation data

In [9]:
# get tf-idf representation of data
tfidf_data = preprocess_tf_idf(corpus, vocab)
test_data = preprocess_tf_idf(test_corpus, vocab)
valid_data = preprocess_tf_idf(valid_corpus, vocab)

In [22]:
# get semantic embeddings of data
w2v_train_data = preprocess_sem_vec(corpus)
w2v_valid_data = preprocess_sem_vec(valid_corpus)
w2v_test_data = preprocess_sem_vec(test_corpus)

### classify a review as negative or positive.

In [11]:
def predict_sentiment(model, doc):
  return model.predict(doc)

### define the model


In [12]:
def define_tf_idf_model(dropout=0.2, l1=1e-5, l2=1e-4):
  model = keras.models.Sequential()
  model.add(layers.Dropout(dropout))
  model.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.L1L2(l1=l1, l2=l2)))
  model.add(layers.Dropout(dropout))
  model.add(layers.Dense(1, activation='sigmoid', kernel_regularizer=regularizers.L1L2(l1=l1, l2=l2)))
  return model

## Running the model

Setting constants

In [19]:
TF_LEARNING_RATE = 3e-5
TF_BATCH_SIZE = 512
TF_EPOCHS = 100

TF_DROPOUT = 0.25
TF_L1 = 5e-6
TF_L2 = 5e-5

In [20]:
# build model
tfidf_model = define_tf_idf_model(dropout=TF_DROPOUT, l1=TF_L1, l2=TF_L2)
tfidf_model.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.SGD(learning_rate=TF_LEARNING_RATE),
    metrics=['accuracy'])
# train model
print("begin training")
tfidf_model.fit(
    tfidf_data,
    taining_labels,
    validation_data=(valid_data, valid_labels),
    epochs=TF_EPOCHS,
    batch_size=TF_BATCH_SIZE)
print("begin evaluation")
tfidf_model.evaluate(test_data, test_labels)

begin training
Epoch 1/100
13586/40000 [=========>....................] - ETA: 35s - loss: 0.7272 - accuracy: 0.5144

KeyboardInterrupt: 

train a model on word2vec embeddings

In [25]:
def define_word2vec_model(dropout=0.4, l1=1e-5, l2=1e-4):
  model = keras.models.Sequential()
  model.add(layers.Dropout(dropout))
  model.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizers.L1L2(l1=l1, l2=l2)))
  model.add(layers.Dropout(dropout))
  model.add(layers.Dense(128, activation='relu', kernel_regularizer=regularizers.L1L2(l1=l1, l2=l2)))
  model.add(layers.Dropout(dropout))
  model.add(layers.Dense(1, activation='sigmoid', kernel_regularizer=regularizers.L1L2(l1=l1, l2=l2)))
  return model

In [26]:
W2V_LEARNING_RATE = 2e-4
W2V_BATCH_SIZE = 512
W2V_EPOCHS = 100

W2V_DROPOUT = 0.1
W2V_L1 = 2e-5
W2V_L2 = 1e-4

In [27]:
# build model
w2v_model = define_word2vec_model(dropout=W2V_DROPOUT, l1=W2V_L1, l2=W2V_L2)
w2v_model.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=W2V_LEARNING_RATE),
    metrics=['accuracy'])
# train model
print("begin training")
w2v_model.fit(
    w2v_train_data,
    taining_labels,
    validation_data=(w2v_valid_data, valid_labels),
    epochs=W2V_EPOCHS,
    batch_size=W2V_BATCH_SIZE)
print("begin evaluation")
w2v_model.evaluate(w2v_test_data, test_labels)

begin training
Epoch 1/100
79/79 [==============================] - 1s 5ms/step - loss: 0.8059 - accuracy: 0.7070 - val_loss: 0.7145 - val_accuracy: 0.7990
Epoch 2/100
79/79 [==============================] - 0s 4ms/step - loss: 0.6342 - accuracy: 0.7951 - val_loss: 0.5574 - val_accuracy: 0.8148
Epoch 3/100
79/79 [==============================] - 0s 4ms/step - loss: 0.5480 - accuracy: 0.8124 - val_loss: 0.5084 - val_accuracy: 0.8326
Epoch 4/100
79/79 [==============================] - 0s 4ms/step - loss: 0.5172 - accuracy: 0.8229 - val_loss: 0.4844 - val_accuracy: 0.8396
Epoch 5/100
79/79 [==============================] - 0s 4ms/step - loss: 0.5017 - accuracy: 0.8276 - val_loss: 0.4733 - val_accuracy: 0.8440
Epoch 6/100
79/79 [==============================] - 0s 4ms/step - loss: 0.4903 - accuracy: 0.8300 - val_loss: 0.4593 - val_accuracy: 0.8500
Epoch 7/100
79/79 [==============================] - 0s 4ms/step - loss: 0.4823 - accuracy: 0.8306 - val_loss: 0.4543 - val_accuracy: 0.847

[0.37983185052871704, 0.8533999919891357]

In [30]:
1e-5 == 10 ** (-5)

True